In [1]:
import ipywidgets as widgets
widgets.Dropdown(options=['One', 'Two', 'Three'], description='Pick:')


Dropdown(description='Pick:', options=('One', 'Two', 'Three'), value='One')

In [10]:
import plotly.io as pio


In [3]:
import plotly.io as pio
pio.renderers.default = 'notebook'


In [8]:
import pandas as pd
import os
from pathlib import Path

# Define the root data directory
root_dir = Path('lipper_data')

# Find all resource_metadata.csv files recursively
csv_files = list(root_dir.rglob('resource_metadata.csv'))

# Read and concatenate all CSV files into one DataFrame
combined_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

# Optionally, print the shape or head to confirm
print(f"Combined {len(csv_files)} files into one DataFrame with shape: {combined_df.shape}")


Combined 22 files into one DataFrame with shape: (7548801, 16)


In [18]:
combined_df['ingestion_dt'] = pd.to_datetime(combined_df['ingestion_dt'], errors='coerce')


In [19]:
print(combined_df['ingestion_dt'].isna().sum(), "rows failed to parse")


9900 rows failed to parse


In [22]:
# Filter and display rows where ingestion_dt is NaT (Not a Time)
na_rows = combined_df[combined_df['ingestion_dt'].isna()]


In [26]:
import pandas as pd

# Ensure ingestion_dt is parsed as datetime
combined_df['ingestion_dt'] = pd.to_datetime(combined_df['ingestion_dt'])

# Extract new columns
combined_df['ingestion_day_of_week'] = combined_df['ingestion_dt'].dt.day_name()
combined_df['ingestion_time'] = combined_df['ingestion_dt'].dt.strftime('%H:%M:%S')
combined_df['ingestion_day_of_month'] = combined_df['ingestion_dt'].dt.day

# Optional: check result


                      ingestion_dt ingestion_day_of_week ingestion_time  \
0 2022-06-25 12:46:53.567000+00:00              Saturday       12:46:53   
1 2022-06-25 12:46:53.567000+00:00              Saturday       12:46:53   
2 2022-06-25 12:46:53.567000+00:00              Saturday       12:46:53   
3 2022-06-25 12:46:53.567000+00:00              Saturday       12:46:53   
4 2022-06-25 12:46:53.567000+00:00              Saturday       12:46:53   

   ingestion_day_of_month  
0                    25.0  
1                    25.0  
2                    25.0  
3                    25.0  
4                    25.0  


In [28]:
combined_df.columns

Index(['schedule_dt', 'supplier_implied_dt', 'ingestion_dt', 'dataset_id',
       'workflow_id', 'frame_id', 'delivery_id', 'latest_health_status',
       'delivery_type', 'resource_id', 'status', 'name', 'url', 'md5_hash',
       'rows', 'size', 'ingestion_day_of_week', 'ingestion_time',
       'ingestion_day_of_month'],
      dtype='object')

In [30]:
combined_df['delivery_type'].unique()

array(['raw', 'delta', nan], dtype=object)

In [31]:
combined_df = combined_df[combined_df['delivery_type'] == 'delta']

In [38]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.io as pio

# Use notebook-compatible renderer
pio.renderers.default = 'notebook_connected'

# Copy your data
df = combined_df.copy()

# Parse time & extract hour in UTC
df['ingestion_time'] = pd.to_datetime(df['ingestion_time'], errors='coerce', utc=True)
df['ingestion_hour'] = df['ingestion_time'].dt.hour

# Set weekday order
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['ingestion_day_of_week'] = pd.Categorical(df['ingestion_day_of_week'], categories=day_order, ordered=True)

# Workflow filter only
workflow_dropdown = widgets.Dropdown(
    options=['All'] + sorted(df['workflow_id'].dropna().unique().tolist()),
    value='All',
    description='Workflow ID:',
    style={'description_width': 'initial'}
)

output = widgets.Output()

def update_dashboard(change=None):
    output.clear_output()

    # Filter by workflow
    filtered = df.copy()
    if workflow_dropdown.value != 'All':
        filtered = filtered[filtered['workflow_id'] == workflow_dropdown.value]

    with output:
        if filtered.empty:
            print("No data for selected workflow.")
            return

        # --- Bar Chart: Ingestions by UTC Hour ---
        hourly_counts = (
            filtered['ingestion_hour']
            .value_counts()
            .sort_index()
            .reset_index()
        )
        hourly_counts.columns = ['ingestion_hour', 'count']

        fig_hour = px.bar(
            hourly_counts,
            x='ingestion_hour',
            y='count',
            labels={'ingestion_hour': 'Hour of Day (UTC)', 'count': 'Ingestions'},
            title='Ingestions by Hour of Day (UTC)'
        )
        fig_hour.update_xaxes(dtick=1)
        fig_hour.show()

        # --- Bar Chart: Ingestions by Day of Week ---
        dow_counts = (
            filtered['ingestion_day_of_week']
            .value_counts()
            .reindex(day_order)
            .dropna()
            .reset_index()
        )
        dow_counts.columns = ['ingestion_day_of_week', 'count']

        fig_dow = px.bar(
            dow_counts,
            x='ingestion_day_of_week',
            y='count',
            labels={'ingestion_day_of_week': 'Day of Week', 'count': 'Ingestions'},
            title='Ingestions by Day of Week'
        )
        fig_dow.show()

        # --- Heatmap: Day of Week × Hour (UTC) ---
        heatmap_df = (
            filtered.groupby(['ingestion_day_of_week', 'ingestion_hour'])
            .size()
            .reset_index(name='count')
        )

        if not heatmap_df.empty:
            fig_heat = px.density_heatmap(
                heatmap_df,
                x='ingestion_hour',
                y='ingestion_day_of_week',
                z='count',
                color_continuous_scale='Blues',
                labels={'ingestion_hour': 'Hour (UTC)', 'count': 'Ingestions'},
                title='Ingestion Heatmap: Day of Week vs Hour (UTC)'
            )
            fig_heat.update_xaxes(dtick=1)
            fig_heat.show()
        else:
            print("No heatmap data to display.")

# Trigger update on dropdown
workflow_dropdown.observe(update_dashboard, names='value')

# Initial dashboard render
update_dashboard()

# Display controls and output
display(workflow_dropdown)
display(output)


/var/folders/zg/psv85v9x1xl6ffh1sc488h7m0000gp/T/ipykernel_35410/1254512608.py:14: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



Dropdown(description='Workflow ID:', options=('All', 'lipper_global_datafeed_globalexusa_lgdf_cxo00221', 'lipp…

Output()